In [32]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
customers_df = pd.read_csv('Customers.csv')
products_df = pd.read_csv('Products.csv')
transactions_df = pd.read_csv('Transactions.csv')

In [25]:
def clean_data():
    # Convert date columns to datetime, handling ISO and mixed formats
    customers_df['SignupDate'] = pd.to_datetime(customers_df['SignupDate'], dayfirst=True, errors='coerce')
    
    # Handle date-time formats for Transactions
    transactions_df['TransactionDate'] = pd.to_datetime(transactions_df['TransactionDate'], dayfirst=True, errors='coerce')
    
    # Remove any rows with missing or invalid dates
    customers_df.dropna(subset=['SignupDate'], inplace=True)
    transactions_df.dropna(subset=['TransactionDate'], inplace=True)
    
    # Remove rows with missing values in other columns
    products_df.dropna(inplace=True)


In [26]:
def analyze_customer_distribution():
    # Analyze customer distribution by region
    region_distribution = customers_df['Region'].value_counts()
    
    # Calculate customer signup trends
    customers_df['SignupMonth'] = customers_df['SignupDate'].dt.to_period('M')
    signup_trends = customers_df['SignupMonth'].value_counts().sort_index()
    
    return region_distribution, signup_trends

In [27]:
def analyze_product_performance():
    # Merge transactions with products
    product_sales = pd.merge(transactions_df, products_df, on='ProductID')
    category_revenue = product_sales.groupby('Category')['TotalValue'].sum().sort_values(ascending=False)
    category_avg_order = product_sales.groupby('Category')['TotalValue'].mean()
    top_products = product_sales.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False)
    return category_revenue, category_avg_order, top_products

In [28]:
def analyze_customer_behavior():
    customer_transactions = pd.merge(transactions_df, customers_df, on='CustomerID')
    region_avg_purchase = customer_transactions.groupby('Region')['TotalValue'].mean()
    region_frequency = customer_transactions.groupby('Region').size()
    customer_value = customer_transactions.groupby('CustomerID')['TotalValue'].sum().sort_values(ascending=False)
    return region_avg_purchase, region_frequency, customer_value

In [29]:
def analyze_temporal_patterns():
    # Add temporal features
    transactions_df['Hour'] = transactions_df['TransactionDate'].dt.hour
    transactions_df['DayOfWeek'] = transactions_df['TransactionDate'].dt.day_name()
    transactions_df['Month'] = transactions_df['TransactionDate'].dt.month
    
    hourly_sales = transactions_df.groupby('Hour')['TotalValue'].sum()
    daily_sales = transactions_df.groupby('DayOfWeek')['TotalValue'].sum()
    monthly_sales = transactions_df.groupby('Month')['TotalValue'].sum()
    return hourly_sales, daily_sales, monthly_sales

In [30]:
clean_data()
region_dist, signup_trends = analyze_customer_distribution()
cat_revenue, cat_avg_order, top_products = analyze_product_performance()
region_avg_purchase, region_frequency, customer_value = analyze_customer_behavior()
hourly_sales, daily_sales, monthly_sales = analyze_temporal_patterns()


In [31]:
print("Key Business Insights:\n")

print("1. Customer Distribution:")
print(region_dist)
print("\n2. Top Product Categories by Revenue:")
print(cat_revenue)
print("\n3. Average Purchase Value by Region:")
print(region_avg_purchase)
print("\n4. Top Selling Products:")
print(top_products.head())
print("\n5. Daily Sales Pattern:")
print(daily_sales)

Key Business Insights:

1. Customer Distribution:
Region
South America    25
North America    20
Europe           17
Asia             17
Name: count, dtype: int64

2. Top Product Categories by Revenue:
Category
Books          192147.47
Electronics    180783.50
Clothing       166170.66
Home Decor     150893.93
Name: TotalValue, dtype: float64

3. Average Purchase Value by Region:
Region
Asia             771.736190
Europe           681.019518
North America    681.350185
South America    715.332800
Name: TotalValue, dtype: float64

4. Top Selling Products:
ProductName
ActiveWear Smartwatch    100
SoundWave Headphones      97
HomeSense Desk Lamp       81
ActiveWear Rug            79
SoundWave Cookbook        78
Name: Quantity, dtype: int64

5. Daily Sales Pattern:
DayOfWeek
Friday        87032.09
Monday       101984.33
Saturday      81399.74
Sunday        92605.93
Thursday      99202.71
Tuesday      108567.61
Wednesday    119203.15
Name: TotalValue, dtype: float64
